In [1]:
import numpy as np
from PIL import Image
import os

from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [2]:
#Geradores de dados
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Geradores
train_gen = train_datagen.flow_from_directory(
    './gender/treino', target_size=(224,224), batch_size=32, color_mode="rgb", class_mode='binary')

val_gen = val_test_datagen.flow_from_directory(
    './gender/validacao', target_size=(224,224), batch_size=32, color_mode="rgb", class_mode='binary')

test_gen = val_test_datagen.flow_from_directory(
    './gender/teste', target_size=(224,224), batch_size=32, color_mode="rgb", class_mode='binary', shuffle=False)

Found 2722 images belonging to 2 classes.
Found 583 images belonging to 2 classes.
Found 586 images belonging to 2 classes.


In [3]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    (Dense(1, activation = "sigmoid"))
])

In [4]:
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

Epoch 1/10
86/86 [==============================] - 47s 502ms/step - loss: 1.1210 - accuracy: 0.8725 - val_loss: 0.8440 - val_accuracy: 0.9074
Epoch 2/10
86/86 [==============================] - 15s 177ms/step - loss: 0.1979 - accuracy: 0.9625 - val_loss: 0.8629 - val_accuracy: 0.9074
Epoch 3/10
86/86 [==============================] - 15s 177ms/step - loss: 0.0717 - accuracy: 0.9875 - val_loss: 0.8332 - val_accuracy: 0.9022
Epoch 4/10
86/86 [==============================] - 15s 178ms/step - loss: 0.0140 - accuracy: 0.9952 - val_loss: 0.7673 - val_accuracy: 0.9022
Epoch 5/10
86/86 [==============================] - 15s 178ms/step - loss: 0.0063 - accuracy: 0.9974 - val_loss: 0.7651 - val_accuracy: 0.9142
Epoch 6/10
86/86 [==============================] - 15s 178ms/step - loss: 0.0071 - accuracy: 0.9974 - val_loss: 0.7869 - val_accuracy: 0.9108
Epoch 7/10
86/86 [==============================] - 15s 178ms/step - loss: 2.4798e-04 - accuracy: 1.0000 - val_loss: 0.7991 - val_accuracy: 0.

In [6]:
loss, acc = model.evaluate(test_gen)
print(f"Acurácia no teste: {acc:.2f}")

19/19 [==============================] - 6s 336ms/step - loss: 0.7156 - accuracy: 0.9130
Acurácia no teste: 0.91


In [7]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Previsões
y_true = test_gen.classes
y_pred = (model.predict(test_gen) > 0.5).astype("int32")

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=['masculino', 'feminino']))

19/19 [==============================] - 3s 153ms/step
[[267  26]
 [ 25 268]]
              precision    recall  f1-score   support

   masculino       0.91      0.91      0.91       293
    feminino       0.91      0.91      0.91       293

    accuracy                           0.91       586
   macro avg       0.91      0.91      0.91       586
weighted avg       0.91      0.91      0.91       586



In [8]:
model.save('modelo_genero_vgg16.keras')